In [1]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from utils import sentences_by_lang, languages, get_trigrams_sets, encode
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

### Przygotowanie danych

In [2]:
# Wczytujemy plik csv tworząc DF z dwoma kolumanmi "lang" oraz "sentence"
csv_file = pd.read_csv('sentences.csv', on_bad_lines='skip', sep='\t', index_col=0, names=["lang", "sentence" ])

In [3]:
# Filtrujemy tabelę, zostawiamy tylko wspierane języki. Dla każdego języka zostawiamy SENTENCES_BY_LANG zdań.
dataset = csv_file[csv_file['lang'].isin(languages)]
results = pd.DataFrame(columns=["lang","sentence"])
for l in languages:
    ds = dataset[dataset["lang"] == l].sample(sentences_by_lang)
    results = pd.concat([results, ds])
results["sentence"] = results["sentence"].str.lower() # pomijamy wielkość liter, aby nie traktować osobno np. "He" i "he"

In [4]:
all_trigrams, lang_trigrams = get_trigrams_sets(results)

### Bag of words

In [5]:
# Tworzymy bag of words, nie wykorzystujemy binarnego bag of words ponieważ trigramy w zdaniu mogą się powtórzyć i stracilibyśmy tę informację.
# Wadą BoW jest fakt, że każdy trigram jest tak samo ważny, ale w naszym problemie to nie przeszkadza.
dic = dict()
for i,t in enumerate(all_trigrams):
    dic[t]=i

vectorizer = CountVectorizer(vocabulary=dic, ngram_range=(3,3), analyzer="char") #ngram_range bierzemy tylko trigramy, analyzer bierzemy pod uwagę znaki,// char_wb nie zliczało poprawnie kolumn

with open('count_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

train_sentences = results["sentence"]
train_langs = results["lang"]
X = vectorizer.fit_transform(train_sentences)
train_features = pd.DataFrame(data=X.toarray(), columns=all_trigrams)
train_min = train_features.min() # najmniejsza wartość z każdej kolumny
train_max = train_features.max() # największa wartość z każdej kolumny
train_features = (train_features - train_min)/(train_max-train_min) # do poprawy bo zwraca NaN, jeśli max value = 0
train_features["lang"] = list(results["lang"]) # dodajemy dodatkową kolumnę z naszym outputem

### Asercje czy wszystko przebiegło pomyślnie, czy w naszym zbiorze nie ma NaN

In [6]:
assert 0 not in train_max, "Jeden z najczęściej występujących trigramów nie wystąpił ani razu"
assert 0 not in (train_max-train_min), "Nie można dzielić przez 0"
assert not train_max.isnull().values.any(),  "NaN w wektorze train_max"
assert not train_min.isnull().values.any(), "NaN w wektorze train_min"
assert not (train_max-train_min).isnull().values.any(), "NaN w mianowniku train_min"
assert not train_features.isnull().values.any(), "NaN w wynikowym DataFrame"

#display(X)
#display(train_features)
#display(train_langs)

### Przygotowanie danych testowych i treningowych 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, train_langs, test_size = 0.2)
encoder = LabelEncoder()
encoder.fit(languages)
with open('encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)
x = train_features.drop('lang',axis=1)
y = encode(train_features['lang'], encoder)

### Przygotowanie modelu

In [8]:
model = Sequential()
model.add(Dense(500, input_dim=len(x.columns), activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### INFO korzystania z GPU lub CPU, można zakomentować jeśli nie chce się korzystać z GPU

In [9]:
from tensorflow.python.client import device_lib
from keras import backend as K

print(device_lib.list_local_devices())

K._get_available_gpus()

print(tf.config.experimental.list_physical_devices())

print("------------------------------------------------------------------------------------------")
physical_devices = tf.config.list_physical_devices('GPU')
try:
  print(physical_devices)
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

#Train model
# Create a MirroredStrategy.
tf.debugging.set_log_device_placement(True)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7117482110917930719
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3667263488
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1236265035702339401
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
------------------------------------------------------------------------------------------
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Uruchomienie treningu

In [ ]:
# Z użyciem gpu
with tf.device('/GPU:0'):
    model.fit(x, y, epochs=1, batch_size=6)
    
# Bez użycia gpu
#model.fit(x, y, epochs=30, batch_size=6)

 36864/133334 [=======>......................] - ETA: 6:26 - loss: 0.0647 - accuracy: 0.9762

### Test precyzji

In [ ]:
labels=model.predict(X_test)
predictions = [encoder.classes_[np.argmax(label)] for label in labels]
accuracy_score(y_test,predictions)

In [ ]:
model.save("trigrams_recognition")
del model